# Using WisardLib

In [1]:
import numpy as np

In this tutorial we show how to encode samples using thermometer encoder, build a simple WNN, train and predict.

Let's create two sets (`X_train`, `y_train`), (`X_test`, `y_test`)

In [2]:
# Train set
X_train = np.array([
    [
        0,    # Sample 0
        20,   # Sample 1
        20,   # Sample 2
        50,   # Sample 3
        255,  # Sample 4
        255,  # Sample 5
        200,  # Sample 6
        189   # Sample 7
    ]  
])

# Class for each train sample
y_train = np.array([
    0, 0, 0, 0, 1, 1, 1, 1
])


# Test set
X_test = np.array([
    15, 192
])

# Class for each test sample
y_test = np.array([0,1])

## Encoding data using thermometer

`wisardlib` has a lot of encoders implemented. Encoders transform data into a `BooleanArray` which is the input for WNNs.
The thermometer encode data in a fixed number of bits depending on the interval of minimum and maximum.

In [3]:
from wisardlib.encoders.thermometer import ThermometerEncoder

number_of_encoding_bits = 4
encoder = ThermometerEncoder(resolution=number_of_encoding_bits)

encoder.fit(X_train)

x_train_encoded = encoder.transform(X_train).squeeze()
x_test_encoded = encoder.transform(X_test).squeeze()

x_train_encoded

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False,  True,  True,  True],
       [False,  True,  True,  True]])

Let's create a WNN model.

In [4]:
from wisardlib.rams.dict_ram import DictRAM
from wisardlib.builder import build_symmetric_wisard

indices = list(range(4))
tuple_size = 2
shuffle = False

model = build_symmetric_wisard(
    RAM_cls=DictRAM,
    RAM_creation_kwargs=None,
    number_of_rams_per_discriminator=2,
    number_of_discriminators=2,
    indices=indices,
    tuple_size=tuple_size,
    shuffle_indices=shuffle
)

In [5]:
model.bleach = 1

In [6]:
model.fit(x_train_encoded, y_train)

Fitting model...: 100%|███████████████████████████| 8/8 [00:01<00:00,  6.03it/s]


WiSARD with 2 discriminators.

In [7]:
y_pred = model.predict(x_test_encoded)
y_pred

Predicting   ...: 100%|███████████████████████████| 2/2 [00:00<00:00, 13.15it/s]


array([[2, 0],
       [0, 2]])

In [8]:
from wisardlib.utils import untie_by_first_class
y_pred, ties = untie_by_first_class(y_pred)
y_pred

Untying...: 100%|██████████████████████████████| 2/2 [00:00<00:00, 13443.28it/s]


array([0, 1])

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

1.0